In [4]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest',
              'ap-northeast-2': '306986355934.dkr.ecr.ap-northeast-2.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the ap-northeast-2 region. You will use the 306986355934.dkr.ecr.ap-northeast-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [55]:
# 데이터를 저장할 S3 버킷을 생성
time = strftime("%y%m%d-%H%M%S")
bucket_name = f'training-job-{time}' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [184]:
from sagemaker import get_execution_role
role = get_execution_role()

bucket = 'passionx3-test-dataset'
subfolder = ''

conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']

my_bucket = 'passionx3-test-dataset'
my_file = 'labeling/job-demo-1/manifests/output/output.manifest'

s3client = boto3.client('s3')
response = s3client.get_object(Bucket=my_bucket, Key=my_file)
objs = response['Body'].read().decode('utf-8').strip().split("\n")

dataset_list = []

for idx, obj in enumerate(objs): 
    item = json.loads(obj) 
    name = f"{idx}.png"
    dataset_list.append({
        "name" : name,
        "source": item['source-ref'],
        "annotation": item['job-demo-1-ref']
    })
#     print(item['source-ref'], item['job-demo-1-ref'])
#     print()

print(dataset_list)


[{'name': '0.png', 'source': 's3://passionx3-test-dataset/imgs/img_1.png', 'annotation': 's3://passionx3-test-dataset/labeling/job-demo-1/annotations/consolidated-annotation/output/0_2021-11-10T03:07:11.553231.png'}, {'name': '1.png', 'source': 's3://passionx3-test-dataset/imgs/img_10.png', 'annotation': 's3://passionx3-test-dataset/labeling/job-demo-1/annotations/consolidated-annotation/output/1_2021-11-10T03:57:04.127238.png'}, {'name': '2.png', 'source': 's3://passionx3-test-dataset/imgs/img_100.png', 'annotation': 's3://passionx3-test-dataset/labeling/job-demo-1/annotations/consolidated-annotation/output/2_2021-11-10T03:04:21.081149.png'}, {'name': '3.png', 'source': 's3://passionx3-test-dataset/imgs/img_101.png', 'annotation': 's3://passionx3-test-dataset/labeling/job-demo-1/annotations/consolidated-annotation/output/3_2021-11-10T03:31:35.163637.png'}, {'name': '4.png', 'source': 's3://passionx3-test-dataset/imgs/img_102.png', 'annotation': 's3://passionx3-test-dataset/labeling/jo